In [117]:
import networkx as nx
import scipy as sp
import numpy as np
import random
import copy
from scipy import sparse
from random import shuffle

from numpy import linalg
from networkx.algorithms.community.label_propagation import label_propagation_communities
import collections

%matplotlib inline

In [111]:
political=nx.read_gml("polblogs.gml",label='id')

#make the graph directed with no parallels
political = nx.DiGraph(political)

#returns a set of nodes of the biggest connected component
largest = max(nx.connected_components(political.to_undirected()), key=len)

# creates the graph of the biggest connected component including direction
CC_max = political.subgraph(largest)


In [143]:
np.seterr(divide='ignore', invalid='ignore')
a = CC_max.copy()

convergence_factor = 0.01

#get the b% of values
percentage=0.1

# re-index node id's to simply proccess, original id's are stored in a label.
CC_max = nx.convert_node_labels_to_integers(CC_max, first_label=0, ordering='default', label_attribute='original_graph_id')

#Returns a list with the value of the nodes
original_values = dict(CC_max.nodes(data='value'))

numbers_of_items_kept_from_percentage = round(len(a)*percentage)

# shuffle the dictionary of values to get the random nodes

shuffled = dict(original_values.items())


shuffled_list = list(shuffled.items())
np.random.shuffle(shuffled_list)
shuffled = dict(shuffled_list)


#keep the first percentage b% of the nodes
nodes_kept = list(shuffled.items())[:numbers_of_items_kept_from_percentage]

nodes_kept_dictionary = dict(nodes_kept)

nodes_to_discover = list(shuffled.items())[numbers_of_items_kept_from_percentage:]

nodes_to_be_discovered_dictionary = dict(nodes_to_discover)

nodes_with_known_value = list(dict(nodes_kept).keys())


#initialize not known node values
        
for key in dict(nodes_kept).keys():
    shuffled[key] = 0.5


#the previous values holds a n x 2 matrix with the original node values
#the first column represent the id the second one the node value
previous_values = np.matrix(list(original_values.items()))

#drop id's so we have a clean n x 1 matrix with the original values
previous_values = np.delete(previous_values, 0, 1)

#the same applies for the new values variable
new_values = np.matrix(list(original_values.items()))

#drop id's
new_values = np.delete(new_values, 0, 1)

#the previous_values and new_values will be updated inside the while loop

# adjacency matrix of the graph
A = nx.adjacency_matrix(CC_max)

# a n x 1 matrix that holds how many neighbors each node has
sum_of_neighbors = np.sum(A,axis=1)

# numbers of nodes
size_of_vector = np.size(sum_of_neighbors,0)


while True:
    
    new_values = A * new_values
    
    #new_values after the multiplication is a n x 1 matrix
    #we need to change the known nodes to their original value

    for key in nodes_kept_dictionary.keys():
        new_values[key] = nodes_kept_dictionary[key]

    #no we need to divide all nodes that we are trying to predict with how many nieghbors they have
    
    for key in nodes_to_be_discovered_dictionary.keys():
        new_values[key] = new_values[key] / sum_of_neighbors[key]
        
    # 1 x n Matrix
    reshape_previous_for_norm = np.array(np.reshape(previous_values,(1, size_of_vector)))
    
    norm_previous = linalg.norm(reshape_previous_for_norm,1)
    
    # 1 x n Matrix
    reshape_next_for_norm = np.array(np.reshape(new_values,(1, size_of_vector)))
    
    norm_new = linalg.norm(reshape_next_for_norm,1)
    
    print(norm_previous)
    print(norm_new)
    break




1.0
9.223372036854776e+18


In [ ]:

#norm_difference = abs(norm_new - norm_of_previous_values)

#print(norm_difference)

#if norm_difference < convergence_factor:
#    break

#previous_values = new_values         
            
